In [1]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
#import pyplot as plt
from functools import reduce
from datetime import date
from datetime import datetime
from dtale.views import startup
import dtale

In [3]:
import matplotlib.rcsetup as rcsetup

In [4]:
dental = pd.read_csv('../data/Smileon.2021.10.17.csv', encoding = "latin-1", skiprows=2)
dental.shape

(57860, 184)

In [5]:
dental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57860 entries, 0 to 57859
Columns: 184 entries, CallReportNum to SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
dtypes: float64(103), int64(6), object(75)
memory usage: 81.2+ MB


In [6]:
#replace spaces with underscores to be able to read  column names
dental.columns = dental.columns.str.replace(' ','_')

In [7]:
#rename columns
dental = dental.rename(columns={'CallerNum': 'Patient'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_Household_Income_Level': 'Household_Income_Level'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_Gender': 'Gender'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_Preferred_Language/Language_Needed': 'Language'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_Race/Ethnicity': 'Race_Ethnicity'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_#_of_People_in_Household': 'People_in_Household'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_Other_Language': 'Other_Language'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_What_other_assistance_programs_are_you_on?': 'What_other_assistance_programs_are_you_on?'})
s = dental[~pd.isnull(dental['Language'])]['Language']

In [8]:
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_#_of_People_in_Household': 'People_in_Household'})
dental = dental.rename(columns={'SMILE_ON_60+_Demographic_Information_-_Other_Language': 'Other_Language'})

In [9]:
dental.drop(['PhoneType', 'OrgNum', 'LinkedToCallReportNum'], axis=1)

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,Patient,PhoneWorkerNum,WasRealCall,WasHangup,WasSexCall,...,SMILE_ON_60+_Screening_-_Treatment_Urgency,SMILE_ON_60+_Screening_-_Untreated_Decay,SMILE_ON_60+_Site_Information_-_Enroller_ID,SMILE_ON_60+_Site_Information_-_Enroller_Location,SMILE_ON_60+_Treatment_Plan_-_Action:_Upload_After_Picture_on_Finish_Tab,SMILE_ON_60+_Treatment_Plan_-_Is_the_treatment_plan_completed?,SMILE_ON_60+_Treatment_Plan_-_Was_a_treatment_plan_developed?,"SMILE_ON_60+_Treatment_Plan_-_Was_the_enrollee_able_to_be_restored_to_function_(can_chew)_and_""social_six""_esthetics_(top_front_six_teeth_are_present_and_disease_free)",SMILE_ON_60+_Verification_of_Eligibility_-_Transportation_Arranged,SMILE_ON_60+_Verification_of_Eligibility_-_Verification_of_Eligibility
0,93115324,SMILE ON 60+ Registration,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57855,55627514,SMILE ON 60+ Clinical,8/1/2018 8:00,8/1/2018 8:08,8,2385621,118821,NaN,NaN,NaN,...,NaN,NaN,Interfaith Dental Clinic - Murfreesboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57856,67251452,SMILE ON 60+ Clinical,5/31/2018 8:27,5/31/2018 8:29,2,2631417,131347,NaN,NaN,NaN,...,NaN,NaN,Friends in Need Health Center - Kingsport,Provider Site,Not available,No,No,Yes,NaN,NaN
57857,53192452,SMILE ON 60+ Registration,5/23/2018 14:11,5/23/2018 14:11,0,-1,60132,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57858,67915561,SMILE ON 60+ Registration,5/15/2018 9:06,5/15/2018 9:10,4,3239813,128174,NaN,NaN,NaN,...,NaN,NaN,Church Health Center - Memphis,Provider Site,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#drops columns that have only NAN values
dental.dropna(how='all', axis=1, inplace=True)

In [11]:
#drop duplicates in Patient (CallerNum)
duplicates = dental.duplicated(['Patient'], keep='first')
dental = dental[~duplicates]

In [12]:
if isinstance(dental, (pd.DatetimeIndex, pd.MultiIndex)):
	dental = dental.to_frame(index=False)

In [13]:
# remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
dental = dental.reset_index().drop('index', axis=1, errors='ignore')
 # update columns to strings in case they are numbers
dental.columns = [str(c) for c in dental.columns] 

In [ ]:
#creating new column - poverty line
filters = [(dental.People_in_Household == 1),
          (dental.People_in_Household == 2),
          (dental.People_in_Household == 3),
          (dental.People_in_Household == 4),
          (dental.People_in_Household == 5),
          (dental.People_in_Household == 6),
          (dental.People_in_Household == 7),
          (dental.People_in_Household == 8),
          (dental.People_in_Household == 9),
          (dental.People_in_Household == 10)]

values = ["17774",
         "24040",
         "30305",
         "36570",
         "42835",
         "49100",
         "55366",
         "61631",
         "61631",
         "61631",
         ]

dental["Poverty_Line"] = np.select(filters, values)
dental.Poverty_Line

In [ ]:
dental['Poverty_Line'] = dental['Poverty_Line'].astype(int)

In [14]:
# too many columns with duplicates, i will merge these values into a couple
# 3 or more
# 4 or more
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; TennCare; SSI/SSDI'],'3 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; TennCare; Section 8'],'3 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; Section 8; SSI/SSDI '],'3 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['TennCare; Section 8; SSI/SSDI'],'3 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; TennCare; Section 8; SSI/SSDI'],'4 +')
# still messy, merge 2 +
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; TennCare'],'2 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; SSI/SSDI'],'2 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['TennCare; SSI/SSDI'],'2 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; Section 8'],'2 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['Section 8; SSI/SSDI'],'2 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['TennCare; Section 8'],'2 +')
dental['What_other_assistance_programs_are_you_on?'] = dental['What_other_assistance_programs_are_you_on?'].replace(['SNAP; Section 8; SSI/SSDI'],'3 +')

In [31]:
dental.drop(dental.index[dental['People_in_Household'] == NaN], inplace=True)

In [32]:
dental = dental.dropna(subset=['People_in_Household'])

In [33]:
dental['People_in_Household'] = dental['People_in_Household'].astype(int)

In [34]:
#drop what is not needed
dental.drop(dental.index[dental['People_in_Household'] == '13900'], inplace=True)
dental.drop(dental.index[dental['People_in_Household'] == '1886'], inplace=True)
dental.drop(dental.index[dental['People_in_Household'] == '1886'], inplace=True)
dental.drop(dental.index[dental['People_in_Household'] == '2864'], inplace=True)
dental.drop(dental.index[dental['People_in_Household'] == '8368'], inplace=True)
dental.drop(dental.index[dental['People_in_Household'] == '112'], inplace=True)

In [35]:
#change value
dental['People_in_Household'] = dental['People_in_Household'].replace(['1_____'],'1')
dental['People_in_Household'] = dental['People_in_Household'].replace(['2_____'],'2')
dental['People_in_Household'] = dental['People_in_Household'].replace(['3_____'],'3')
dental['People_in_Household'] = dental['People_in_Household'].replace(['6_____'],'6')
dental['People_in_Household'] = dental['People_in_Household'].replace(['1.0'],'1')
dental['People_in_Household'] = dental['People_in_Household'].replace(['2.0'],'2')
dental['People_in_Household'] = dental['People_in_Household'].replace(['3.0'],'3')
dental['People_in_Household'] = dental['People_in_Household'].replace(['4.0'],'4')
dental['People_in_Household'] = dental['People_in_Household'].replace(['5.0'],'5')
dental['People_in_Household'] = dental['People_in_Household'].replace(['6.0'],'6')
dental['People_in_Household'] = dental['People_in_Household'].replace(['7.0'],'7')
dental['People_in_Household'] = dental['People_in_Household'].replace(['8.0'],'8')
dental['People_in_Household'] = dental['People_in_Household'].replace(['9.0'],'9')
dental['People_in_Household'] = dental['People_in_Household'].replace(['10.0'],'10')
dental['People_in_Household'] = dental['People_in_Household'].replace(['0'],'10')

In [54]:
#number value 0, 9, and 10 were lumped with 8 since the count was very low
dental['People_in_Household'] = dental['People_in_Household'].replace(['9'],'8')
dental['People_in_Household'] = dental['People_in_Household'].replace(['10'],'8')

In [52]:
#remove $ and lump everything after 50k
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$0 - $21 280'],'0 - 21280')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$21246-$24280'],'21246 - 24280')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$24281-$27315'],'24281 - 27315')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$28806-$32920'],'28806 - 32920')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$32921-$37035'],'32921 - 37035')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$41561-$46755'],'41561 - 46755')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$36366-$41560'],'36366 - 41560')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$43926-$50200'],'43926 - 50200')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$50201-$56475'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$59046-$67480'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$58841-$66195'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$51486-$58840'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$76121-$85635'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$84761-$95355'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$67481-$75915'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$66606-$76120'],'50000 - 85000')
dental['Household_Income_Level'] = dental['Household_Income_Level'].replace(['$74166-$84760'],'50000 - 85000')

In [38]:
dental.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10211 entries, 0 to 10710
Columns: 130 entries, CallReportNum to SMILE_ON_60+_Verification_of_Eligibility_-_Verification_of_Eligibility
dtypes: float64(49), int32(1), int64(6), object(74)
memory usage: 10.4+ MB


In [55]:
#Average of people in a household?
# 'SMILE_ON_60+_Demographic_Information_-_#_of_People_in_Household''
householdnumber = dental['People_in_Household'].value_counts()
print(householdnumber)

1     6446
2     2733
3      507
4      260
5      153
6       57
7       32
8       15
9        5
10       3
Name: People_in_Household, dtype: int64


In [53]:
#obtain Household Income Level
#'SMILE_ON_60+_Demographic_Information_-_Household_Income_Level'
houseincome = dental['Household_Income_Level'].value_counts()
print(houseincome)

0 - 21280        7348
21246-24280      1354
24281-27315       432
28806-32920       390
32921-37035       145
41561-46755        79
36366-41560        70
50000 - 85000      47
43926-50200        22
Name: Household_Income_Level, dtype: int64


In [66]:
# https://seaborn.pydata.org/examples/part_whole_bars.html

sns.set_theme(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(15, 12))


# Plot the total crashes
sns.set_color_codes("pastel")
sns.barplot(x='Poverty_Line', y="People_in_Household", data=dental,
            label="Household Income", color="c")

# Plot the crashes where alcohol was involved
sns.set_color_codes("muted")
sns.barplot(x="Household_Income_Level", y="People_in_Household", data=dental,
            label="Poverty Line", color="b")

# Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 9), ylabel="",
xlabel="{Household_Income_Level}")
sns.despine(left=True, bottom=True)

In [68]:
#Enrolled in OTHER assistance programs, which ones?
# 'SMILE_ON_60+_Demographic_Information_-_What_other_assistance_programs_are_you_on?'
otherprograms = dental['What_other_assistance_programs_are_you_on?'].value_counts()
print(otherprograms)

SSI/SSDI     1962
2 +          1099
SNAP          729
3 +           460
TennCare      433
Section 8     175
4 +            92
Name: What_other_assistance_programs_are_you_on?, dtype: int64


In [ ]:
#Plt.hist(People_in_Household, bins=8)
#Plt.xlabel('name')
#Plt.ylabel('name')
#Plt.show()

In [69]:
d = dtale.show(dental)
# open it in a new window in browser
d.open_browser()